# import

In [184]:
import numpy as np
import pandas as pd

import mdptoolbox
import time as time
import itertools
import csv
from decimal import *
import pdb

# Execute main functions

In [185]:
#read main functions 

#Transition Probability Matrix
transProb=open(r"E:\one-Drive\OneDrive - Auburn University\Auburn University\RA\GMS\8-First Contribution\Main Codes\Transition Probability Matrix.py",encoding="utf8")
read_transProb=transProb.read()
exec(read_transProb)

#Rewards
Rewards=open(r"E:\one-Drive\OneDrive - Auburn University\Auburn University\RA\GMS\8-First Contribution\Main Codes\Rewards.py",encoding="utf8")
read_Rewards= Rewards.read()
exec(read_Rewards)

#Find Optimal Policies
FOP=open(r"E:\one-Drive\OneDrive - Auburn University\Auburn University\RA\GMS\8-First Contribution\Main Codes\Find Optimal Policies.py",encoding="utf8")
read_FOP= FOP.read()
exec(read_FOP)

#Make a dictionary of all combinations and strategies
MDCS=open(r"E:\one-Drive\OneDrive - Auburn University\Auburn University\RA\GMS\8-First Contribution\Main Codes\Make a dictionary of all combinations and strategies.py",encoding="utf8")
read_MDCS= MDCS.read()
exec(read_MDCS)

# DoE

In [186]:
#range of sequencing cost
q=np.arange(0,-627,-1)  #range of numbers for seq cost cost in array
range_seqCost=q.tolist() #range of numbers for seq cost in list
#range_seqCost

In [187]:

def doe_sequencingCost(range_seqCost,z1,z2):

    
    P=newTransProb(z1, z2)   #Transition Probability MAtrix
    
    transitions=P.reshape(5,19,19)   #reshape the transtion probability matrix to (A,S,S)
 
    
    
    #make a dictionary
    value=[]
    sCost=dict.fromkeys(range_seqCost,value)  #make dictionary for sequencing cost
    General_keys1= sCost.keys()
    General_keys=list(General_keys1)  #list of all keys
    
    
    #for loop
    for k in General_keys:
        SeqCost=k

        #rewards
        R=rewards(SeqCost=k)    #do not forget to put SeqCost=k in th function 
        reward=R.reshape(19,5)
        # Total cost use policy iteration method
        discount=0.9999
        pi=mdptoolbox.mdp.PolicyIteration(transitions, reward, discount=0.9999, policy0=None, max_iter=10000, eval_type=0)
           # four 9s
        #pi=mdptoolbox.mdp.ValueIteration(transitions, reward, discount, max_iter=10000)
        #run the  Policy iteration
        pi.run() 

        total_cost=pi.V
        rounded_total_cost=round(total_cost[0],2)
        #put all optimal Policies for Policy Iteration
        policy=pi.policy
        result=[]
        
        #values
        #result.append(rounded_total_cost)
        result.append(policy)
        #put all total costs and policies as values in dictionary
        sCost[SeqCost]=result     #do not use append and extend here


    return sCost

In [188]:
#assign a sspecific variable to this experiment
dsc=doe_sequencingCost(range_seqCost,z1=0.9,z2=0.1)          
#dsc

# Find Optimal Policies

In [189]:
#it calls the function from Main Functions script
fuop=dsc  #fuop is a constant name for a variable which gets the experiments
opt_dict1=find_optimal_policies(fuop)   #the input is Fuop output is opt_dict
opt_dict1

{'No-No': [],
 'No-Gel': [],
 'Gel-No': [[(0, 2, 2, 1, 1, 1, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 0)]],
 'Gel-Gel': [[(0, 2, 2, 2, 1, 1, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 0)],
  [(0, 2, 2, 2, 2, 1, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 0)]],
 'No-Bio': [],
 'Bio-No': [],
 'Bio-Bio': [],
 'Seq-No': [[(0, 4, 1, 1, 1, 1, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 0)],
  [(0, 4, 2, 1, 1, 1, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 0)]],
 'Seq-Gel': [],
 'Seq-Bio': [],
 'Bio-Gel': [],
 'Gel-Bio': []}

# Make a dictionary of all combinations and strategies

In [190]:
##it calls the function from Main Functions script
dict_NGBS1=make_dict_comb_stra(fuop,opt_dict1)  
#dict_NGBS1

# Strategies and Combinations in a dataframe

In [191]:
# convvert dict_NBGS to list 
list_NBGS=list(dict_NGBS1.items())

#seperate lists
SeqCost=[]
insp_stra=[]
for i in range(len(list_NBGS)):
    SeqCost.append(list_NBGS[i][0])
    insp_stra.append(list_NBGS[i][1])
    
# make a dataframe from NBGS
dt_NBGS=pd.DataFrame(list(zip(SeqCost,insp_stra)),columns=["Sequencing Cost",'Inspection Strategy'])
#show all combination and corresponding optimal policy in one table
#pd.set_option('max_columns', None)  #show all columns
#pd.set_option('max_rows', None)     #show all rows
#dt_NBGS

In [192]:
# Show the specific (Let's make life easier)
dt_NBGS1=dt_NBGS.copy()

In [193]:
dt_NBGS1["Seq-No"]=" "
dt_NBGS1['No-No']=" "
dt_NBGS1['No-Gel']=" "
dt_NBGS1["Gel-Gel"]=" "

dt_NBGS1["Gel-Bio"]=" "
dt_NBGS1["Gel-No"]=" "
dt_NBGS1["No-Bio"]=" "
dt_NBGS1["Bio-Gel"]=" "

dt_NBGS1["Bio-Bio"]=" "
dt_NBGS1["Bio-No"]=" "
#dt_NBGS1

In [194]:
for i in range(0, len(dt_NBGS1['Inspection Strategy'])):
    if dt_NBGS1['Inspection Strategy'][i]=="Seq-No":
            dt_NBGS1["Seq-No"][i]=dt_NBGS1['Sequencing Cost'][i]

    elif dt_NBGS1['Inspection Strategy'][i]=='No-No':
            dt_NBGS1['No-No'][i]=dt_NBGS1['Sequencing Cost'][i]

    elif dt_NBGS1['Inspection Strategy'][i]=='No-Gel':
            dt_NBGS1['No-Gel'][i]=dt_NBGS1['Sequencing Cost'][i]
            
    elif dt_NBGS1['Inspection Strategy'][i]=='Gel-Gel':
            dt_NBGS1['Gel-Gel'][i]=dt_NBGS1['Sequencing Cost'][i]        

    elif dt_NBGS1['Inspection Strategy'][i]=="Gel-Bio":
            dt_NBGS1["Gel-Bio"][i]=dt_NBGS1['Sequencing Cost'][i]

    elif dt_NBGS1['Inspection Strategy'][i]=="Gel-No":
            dt_NBGS1["Gel-No"][i]=dt_NBGS1['Sequencing Cost'][i]

    elif dt_NBGS1['Inspection Strategy'][i]=="No-Bio":
            dt_NBGS1["No-Bio"][i]=dt_NBGS1['Sequencing Cost'][i]

    elif dt_NBGS1['Inspection Strategy'][i]=="Bio-Gel":
            dt_NBGS1["Bio-Gel"][i]=dt_NBGS1['Sequencing Cost'][i]

    elif dt_NBGS1['Inspection Strategy'][i]=="Bio-Bio":
            dt_NBGS1["Bio-Bio"][i]=dt_NBGS1['Sequencing Cost'][i]

    elif dt_NBGS1['Inspection Strategy'][i]=="Bio-No":
            dt_NBGS1["Bio-No"][i]=dt_NBGS1['Sequencing Cost'][i]

    else:
        print("Something is wrong")

C:\TEMP/ipykernel_13656/2651748627.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dt_NBGS1["Gel-No"][i]=dt_NBGS1['Sequencing Cost'][i]
C:\TEMP/ipykernel_13656/2651748627.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dt_NBGS1['Gel-Gel'][i]=dt_NBGS1['Sequencing Cost'][i]
C:\TEMP/ipykernel_13656/2651748627.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dt_NBGS1["Seq-No"][i]=dt_NBGS1['Sequencing Cost'][i]


## Export to csv

In [195]:
#dt_NBGS1
dt_NBGS1.to_csv('my_dataframe.csv', index=False)